In [78]:
import pandas as pd
import numpy as np
import pdfplumber
import re
import nltk
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# dictionary from excel file

In [2]:
excell = pd.read_excel('PWC.xlsx', sheet_name = 'Resume' )

In [3]:
# excell.head(59)

In [4]:
# excell.iloc[28,:]

In [5]:
# commission = {'abanca': {'1':{'Emissão de extrato':{'1.1':'Mensal','1.2':')utros, para além do indicado em 1.1'}},'2':'Fotocópias de segundas vias de talões de depósito',\
#                         '3': 'Manutenção de conta', '4': {'Levantamento de numerário':{'4.1':'Ao balcão, com apresentação de cheque',\
#                                                                                       '4.2': 'Ao balcão, sem apresentação de cheque'},\
#                                                          }, '5': 'Adesão ao serviço de banca à distância', '6':'Depósito de moedas metálicas',\
#                         '7':'Alteração de titulares'}, \
#               'ctt':{'1':'Extrato integrado mensal', '2':'Consulta de saldo de conta DO com comprovativo',\
#                      '3':'consulta de movimentos de conta do com comprovativo', '4': 'Emissão 2as Vias de Avisos e Outros Documentos',\
#                      '5':'Alteração de titularidade'},
#               'bic':{'1':'Manutenção de conta','2':'Levantamento de numerário','3':'Levantamento usd em contas usd',\
#                     '4': 'Extracto integrado','5':'Extracto avulso'}}

In [6]:
commission = {'abanca': {'1':'Emissão de extrato','2':'Fotocópias de segundas vias de talões de depósito',\
                        '3': 'Manutenção de conta', '4': 'Levantamento de numerário', '5': 'Adesão ao serviço de banca à distância', '6':'Depósito de moedas metálicas',\
                        '7':'Alteração de titulares'}, \
              'ctt':{'1':'Extrato Integrado mensal', '2':'Emissão 2ªs Vias de Avisos e Outros Documentos',\
                     '3':'Comissão de manutenção de conta','4':'Levantamento de Numerário ao Balcão',
                     '5':'Comissões por descoberto bancário','6':'Consulta de Movimentos de conta do com comprovativo', '7': 'Consulta de Saldo de conta DO com comprovativo',\
                     '8':'Alteração de titularidade'},
              'bic':{'1':'Manutenção de conta','2':'Levantamento de numerário','3':'Levantamento usd em contas usd',\
                    '4': 'Extracto integrado','5':'Extracto avulso'}}

# extract pdf

In [13]:
def len_sentences(x):
    if len(nltk.sent_tokenize(x))== 1:
        return x.replace('\n','. ')
    return x.replace('\n','')

In [14]:
def extract_clean(pdf,n_page, splitting_right=False):
    file = pdfplumber.open(pdf)
    page = file.pages[n_page]
    text = page.extract_text()
    text = text.replace('Isento', '0,00')
    text = text.replace('n/a', str(np.nan))
    text = re.sub('--', str(np.nan), text)
    if splitting_right == True:
        text = text.replace('\n','. ')
#         text = len_sentences(text)
        text = nltk.sent_tokenize(text)
        return text
    text = text.replace('\n',' ')
#     text = len_sentences(text)
    text = nltk.sent_tokenize(text)
    return text

In [15]:
abanca = extract_clean('data/ABANCA2021com.pdf', 4)
ctt = extract_clean('data/BancoCTT2021.pdf', 4, splitting_right=True)
bai = extract_clean('data/bancoBAIeurope2021com.pdf', 4)
bankinter = extract_clean('data/bankinter.pdf', 4)

# getting file clean and corrispondent part in dictionary

In [16]:
# abanca
# bai
ctt
# bankinter

['Entrada em Vigor: 01-Jan-2021.',
 '1.',
 'CONTAS DE DEPÓSITO (PARTICULARES) (ÍNDICE).',
 '1.1.',
 'Depósitos à ordem.',
 'Comissões.',
 'Acresce .',
 'Euros  Outras condições.',
 'Valor Anual Imposto.',
 '(Mín/Máx).',
 'Contas de Depósito à Ordem (Comissões aplicáveis a todas as Contas).',
 'Comissões durante a vigência do contrato.',
 '1.Extrato Integrado mensal 0,00 0,00 nan nan.',
 '2.Consulta de Saldo de conta DO com comprovativo 0,50 nan IS - 4% Comissão por emissão ao balcão (1).',
 '3.Consulta de Movimentos de conta DO com.',
 '2,00 nan IS - 4% Comissão por emissão ao balcão.',
 'comprovativo.',
 '4.Emissão 2ªs Vias de Avisos e Outros Documentos 5,00 nan IVA - 23% Comissão por emissão ao balcão.',
 '5.Emissão 2ª Via - Extrato Integrado 10,00 nan IVA - 23% Comissão por emissão ao balcão.',
 'Conta Banco CTT .',
 'Comissões durante a vigência do contrato.',
 '1.Comissões por descoberto bancário vide Secção 2.5.',
 'Descobertos bancários.',
 '2.Levantamento de Numerário ao Balcão

In [17]:
def search_com(x, pdf, page,splitting_right=False):
    decod = {}
    if x not in commission:
        return 'not in the dictionary'
    decod = commission[x]
    file = extract_clean(pdf, page,splitting_right=False)
    return decod, file

In [199]:
ctt_here = search_com('ctt', 'data/BancoCTT2021.pdf', 4,splitting_right=True)
ctt_here[1]

'o'

In [144]:
abanca_here = search_com('abanca', 'data/ABANCA2021com.pdf', 4)
bai_here = search_com('bai','data/bancoBAIeurope2021com.pdf', 4)

- **try with module from scraping.py**

In [20]:
from scraping import Scraping

In [184]:
abanca = Scraping('data/ABANCA2021com.pdf', 4).search_com('abanca')
abanca[1]

['Entrada em vigor: 20-Nov-2020    1.',
 'CONTAS DE DEPÓSITO (PARTICULARES)                                                                                   ÍNDICE   1.1 Depósitos à ordem  Comissões  Acresce  Outras    Euros (Mín/Máx)  Valor anual  Imposto  condições  Produtos: Fora de comercialização: Conta D.O.',
 'Particulares; Conta Ordenado; Conta Standard; Conta Future; Conta  Kids; Conta Global  Em comercialização: Conta Base; Conta Private; Conta Value; Conta Smart; Conta Futuro; Conta Serviços Mínimos  Bancários, Conta Moeda Estrangeira e Conta ABANCA Internacional.',
 'As comissões abaixo indicadas ocorrem no momento em que se verifica a operação e aplicam-se às contas acima identificadas, salvo  nos casos em que expressamente se exclui.',
 '1.',
 'Emissão de extrato      1.1  Mensal (enviado ao domicílio)  0,00  nan  IVA – 23%  1.2  Outros, para além do indicado em 1.1  2,00 por extrato  1.3  2ª Via      7,50    2.',
 'Fotocópias de segundas vias de talões de depósito   5,0

In [230]:
def n_account(text, splitting_right = False):
    accounts = []
    finals = []
    if splitting_right == True:
        for sentence in text:
            if 'Conta' in sentence:
                finals.append(sentence)
        return finals
    for sentence in text:
        contas = re.split('Conta',sentence)
        accounts.append(contas)
    for account in accounts:
        if len(account)>1:
            account = ['Conta'+ x for x in account]
            finals.append(account)
            
            
    return finals

In [221]:
# n_account(ctt_here)

[]

# Searching for lines with values of dictionary inside

In [68]:
def values(x):
    file = x[1]
    names = x[0]
    lista = {}
    keys = [x for x in names.keys()]
    for key in keys:
            for ind,sentence in enumerate(file):
                if names[key] in sentence:
                    if key in lista:
                        if '[0-9]{1-2},[0-9]{2}' in sentence:
                            lista[names[key]]= lista[key].append(sentence)
                        else:
                            lista[names[key]]= lista[key].append(' '.join([sentence,file[ind+1]]))
                    else:
                        if '[0-9]{1-2},[0-9]{2}' in sentence:
                            lista[names[key]]= [sentence]
                        else:
                            lista[names[key]]= [' '.join([sentence,file[ind+1]])]
    return lista

In [149]:
# ctt_here[0]
# values(bai_here)

In [150]:
# values(ctt_here)

In [89]:
commission = {'abanca': {'1':'Emissão de extrato','2':'Fotocópias de segundas vias de talões de depósito',\
                        '3': 'Manutenção de conta', '4': 'Levantamento de numerário', '5': 'Adesão ao serviço de banca à distância', '6':'Depósito de moedas metálicas',\
                        '7':'Alteração de titulares'}, \
              'ctt':{'1':'Extrato Integrado', '2':'Emissão 2ªs Vias de Avisos e Outros Documentos',\
                     '3':'Comissão de manutenção de conta','4':'Levantamento de Numerário ao Balcão',
                     '5':'Comissões por descoberto bancário','6':'Consulta de Movimentos de conta DO com', '7': 'Consulta de Saldo de conta DO com comprovativo',\
                     '8':'Alteração de titularidade'},
              'bai':{'1':'Manutenção de conta','2':'Levantamento de numerário','3':'Levantamento usd em contas usd',\
                    '4': 'Extracto integrado','5':'Extracto avulso'}}

In [92]:
from scraping import Scraping

In [146]:
bai = Scraping('data/bancoBAIeurope2021com.pdf', 4).values('bai')
abanca = Scraping('data/ABANCA2021com.pdf', 4).values('abanca')

In [148]:
bai

{'Manutenção de conta': ['Manutenção de conta 4,10 €/por Ano 4,10 € I.S. - 4% Nota (2) Comissões Descoberto Bancário  vide Secção 2.5 Descobertos bancários Outras despesas associadas nan.'],
 'Levantamento de numerário': ['Levantamento de numerário 20,00 € 0,5% Incide sobre o montante do  3. Levantamento USD em contas USD I.S.'],
 'Extracto integrado': ['Extracto integrado 0,00 Periodicidade: Mensal  5,00 €/ Moeda Estrangeira  5. Extracto avulso I.S.'],
 'Extracto avulso': ['Extracto avulso I.S. - 4% A pedido do cliente.']}

In [138]:
# abanca['Emissão de extrato'][0].split(r"(\d),(\d) | (\d)")

['Emissão de extrato      1.1  Mensal (enviado ao domicílio)  0,00  nan  IVA – 23%  1.2  Outros, para além do indicado em 1.1  2,00 por extrato  1.3  2ª Via      7,50    2. Fotocópias de segundas vias de talões de depósito   5,00 por documento  nan  IVA – 23%    3.']

In [228]:
ctt_new = Scraping('data/BancoCTT2021.pdf', 4,splitting_right=True).search_com('ctt')[1]
bai_new = Scraping('data/bancoBAIeurope2021com.pdf', 4).search_com('bai')[1]
abanca_new = Scraping('data/ABANCA2021com.pdf', 4).search_com('abanca')[1]

In [232]:
n_account(ctt_new, splitting_right = True) 
n_account(abanca_new)
n_account(bai_new)

[['ContaDepósitos à ordem Comissões Acresce  Outras condições  (Mín/Máx) Valor Anual Imposto ',
  'Conta Depósito à Ordem (EUR/ Moeda Estrangeira) Comissões vigência contrato  5,00 €/ Moeda Estrangeira  60,00 €/ Moeda  1.'],
 ['Contapor cada Comissões Descoberto Bancário  vide Secção 2.5 Descobertos bancários ',
  'Conta de Serviços Mínimos Bancários Comissões vigência contrato 1.'],
 ['ContaNota (2) ',
  'Conta de Serviços Minímos Bancários: Condições de acesso: Particulares residentes num Estado Membro da União Europeia que reúnam qualquer uma das seguintes condições: I. Não sejam titulares de qualquer conta de depósito à ordem junto do Banco BAI Europa, S.A. (BAIE) ou de qualquer outra Instituição de Crédito; ou  II.Sejamtitularesdeumaúnicacontadedepósitoàordemnosistemabancárioetenhamsidonotificadosdequeamesmaserá encerrada; ou III.'],
 ['ContaO acesso à ',
  'Conta de Serviços Mínimos Bancários não depende da aquisição de outros produtos ou serviços.'],
 ['ContaBanco BAI Europa, S.

In [235]:
Scraping('data/ABANCA2021com.pdf', 4, 'abanca').n_account()
Scraping('data/BancoCTT2021.pdf', 4,'ctt',splitting_right=True).n_account()

['Contas de Depósito à Ordem (Comissões aplicáveis a todas as Contas).',
 'Conta Banco CTT .',
 'Conta Júnior (Montante Mínimo de Abertura 25,00€).',
 'Conta Base (Montante Mínimo de Abertura 100,00€).',
 'Conta de Serviços Mínimos Bancários .',
 'Banco CTT, S.A. Contas de Depósito-Particulares - Pág.1/2']